# FDA Reading Room Dataframe Tidying

In [25]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
import lxml
import html5lib 
from bs4 import BeautifulSoup
import janitor
import tempfile
import os # to get current opperating system
import shutil
import time

import cv2
from PIL import Image
from pathlib import Path
from pdf2image import convert_from_path

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ncruickshank\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

# selenium functions
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #allow  you to enter keystrokes into fields
from selenium.webdriver.support.ui import Select #allow you to select a dropdown item
from selenium.webdriver.support.ui import WebDriverWait #lets you modify a field before proceeding
from selenium.common.exceptions import NoSuchElementException 

import nltk
import re

In [3]:
reading_room = pd.read_csv("data/FDA Reading Room Form 483 Dataset - Last Updated 20210512.csv")

## Trim one set of observations to figure out the syntax
Test case is Edge Pharma Form 483 from 03/30/2020.

### Pre-process test and separate into different OBSERVATIONS

In [114]:
# isolate test case
test = reading_room.observations[11] # 11 = edge pharma

# cut the preamble before OBSERVATION 1 and strip leading and lagging whitespace
preamble = 'This document lists observations made by the FDA representative(s) during the inspection of your facility. They are inspectional observations, and do not represent a final Agency determination regarding your compliance. If you have an objection regarding an observation, or have implemented, or plan to implement, corrective action in response to an observation, you may discuss the objection or action with the FDA representative(s) during the inspection or submit this information to FDA at the address above. If you have any questions, please contact FDA at the phone number and address above.  DURING AN INSPECTION OF YOUR FIRM WE OBSERVED: '
str_start = test.find(preamble) + len(preamble)
test = test[str_start:].strip()

# replace carraige returns with regular whitespace
test = re.sub(r"(\n|\x0c)", " ", test).strip()

# split the text into different strings based on OBSERVATIONS
observations = re.split(r"OBSERVATION [0-9]+\s+", test)
cleaned_observations = []
for observation in observations:
        
    # split observation into words
    tokens = nltk.word_tokenize(observation)
    
    # removeall non-alphabetic words and convert to lower case
    words# = [word.lower() for word in tokens if word.isalpha()]
    
    # filter out stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [word.lower() for word in words if not word in stop_words]
    
    # stem the verbs
    #porter = nltk.PorterStemmer()
    #stemmed = [porter.stem(word) for word in words]
    
    def list_to_string(s):
        str1 = " "
        return (str1.join(s))
    
    clean_observation = list_to_string(words)
    
    cleaned_observations.append(clean_observation)

https://stackoverflow.com/questions/33098040/how-to-use-word-tokenize-in-data-frame

In [116]:
cleaned_observations